# Data Science

Debido al requerimiento del cliente, se procederan a reordenar la data, para obtener un set de datos horarios donde cada fila represente la cantidad de viajes realizados en esa hora en especifico(hora de comienzo del viaje).

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('Data\process_taxis.parquet',engine='pyarrow')

In [ ]:
df.set_index(keys='pickup_datetime',inplace=True)

In [ ]:
df.index

In [ ]:
# Crear un nuevo DataFrame con la cantidad de registros por hora
df_hour = df.resample('H').size().to_frame('viajes')

In [ ]:
df_hour.tail(2)

In [ ]:
df.columns

### <center>Models</center>

Para cumplir con las expectativas del cliente, se evaluaran 2 modelos

- Forecaster Autoregresive: 

Para modelar la evolución de los datos como serie de tiempo, se optó por un modelo Forecaster Autoregresivo que incorpora lags de la variable objetivo y variables de tiempo (como la hora, mes, dia, etc). Esta elección se fundamenta en la hipótesis de que los valores futuros de la serie están fuertemente influenciados por sus valores pasados y por patrones estacionales o cíclicos asociados a las variables de tiempo.

Los lags permiten capturar la autocorrelación presente en la serie, mientras que las variables de tiempo ayudan a modelar los componentes estacionales y de tendencia, lo que resulta en predicciones más precisas y robustas.
Luego de esto se evaluara el desempeño del modelo para su comparacion.

- Modelo de Predicción en Cascada(Autoregresion y Random Forest o regresion lineal multiple):

Para realizar predicciones futuras de la variable objetivo, el modelo se desarrollará en dos etapas. En la primera etapa, se utilizarán modelos predictivos para estimar los valores futuros de las variables auxiliares (ganancia neta, duración del viaje, y CO₂ emitido). Estas estimaciones serán generadas únicamente a partir de valores pasados de cada una de las variables auxiliares, aplicando un modelo autoregresivo puro. Así, para cada variable auxiliar, se entrenará un modelo autoregresivo independiente que capture las tendencias y patrones de su serie temporal individual sin depender de otras variables.

En la segunda etapa, las estimaciones futuras de estas variables auxiliares se utilizarán como entradas (features) en el modelo final. Este modelo tendrá como objetivo principal la predicción de la variable objetivo y empleará tanto los valores estimados de las variables auxiliares como el valor pasado de la variable objetivo para optimizar su precisión en las predicciones futuras. 

******

### <center>Target</center>

El desafio al cual nos ha sometido el cliente, es predecir la demanda de taxis, para lo cual es necesario saber o estimar un valor de cuantos viajes pudieran llegar a realizarse. Por lo cual, se utilizara como target la columna "viajes" creada a partir del recuento de instancias de nuestra data principal transformada en un dataframe con frecuencia horaria.



In [ ]:
#sum_for_price = df['total_price'].resample('H').sum().to_frame('total_price')
#sum_for_cost = df['fuel_cost'].resample('H').sum().to_frame('fuel_cost')
sum_for_profit = df['net_profit'].resample('H').sum().to_frame('net_profit')
sum_for_duration = df['trip_duration'].resample('H').sum().to_frame('trip_duration')
sum_for_co2 = df['co2'].resample('H').sum().to_frame('co2')

In [ ]:
# Unir los resultados en un solo DataFrame
resultado = df_hour.join([sum_for_profit,sum_for_duration,sum_for_co2])

In [ ]:
resultado

In [ ]:
df.reset_index(inplace=True)
df['year'] = df['pickup_datetime'].dt.year
df['quarter'] = df['pickup_datetime'].dt.quarter
df['month'] = df['pickup_datetime'].dt.month
df['day_month'] = df['pickup_datetime'].dt.day
df['weekday'] = df['pickup_datetime'].dt.day_name()
df['hour'] = df['pickup_datetime'].dt.hour
df['weekend'] = df['pickup_datetime'].apply(lambda x: x.weekday() < 5).astype(int) # 1 = weekday, 0 = weekend
#df.drop(columns=['pickup_datetime','dropoff_datetime'],inplace=True)